In [ ]:
import numpy as np
import pylab as pl
%matplotlib inline
from PyEMD import EMD, EEMD
from vmdpy import VMD 

data = np.loadtxt('bitcoin_data.txt')

def unit(x):
    return (x - np.min(x))/(np.max(x) - np.min(x))

# candidate alpha values : [0.1*i for i in range(1,10)]

# Encoding of CCD
def weist(data, alpha):
    def unit(x):
        return (x - np.min(x))/(np.max(x) - np.min(x))
    data = unit(data)
    beta = 0.5**(1-alpha)
    Data = []
    for i in range(len(data)):
        if data[i] < 1/2:
            Data.append( beta * (data[i]**alpha) )
        else:
            Data.append( ( (1/beta)**(1/alpha) ) * ( (data[i]-0.5)**(1/alpha) ) + 0.5)
    return np.array(Data)

# Decoding of CCD
def inv_weist(prediction, original_data, alpha):
    beta = 0.5/((0.5)**alpha)
    M, N = np.max(original_data), np.min(original_data)
    pred = []
    for i in range(len(prediction)):
        if prediction[i] < 1/2:
            pred.append( (1/beta*prediction[i])**(1/alpha) )
        else:
            pred.append( beta*( (prediction[i]-0.5)**(alpha) ) + 0.5  )
    return np.array(pred)

# EMD
def empirical_mode_decomposition(F):
    emd = EMD()
    emd.emd(F)
    imfs, res = emd.get_imfs_and_residue()
    return np.concatenate([imfs.T, res.reshape(-1,1)], axis=1)

# EEMD
def ensemble_empirical_mode_decomposition(F):
    eemd = EEMD()
    eemd.eemd(F)
    imfs, res = eemd.get_imfs_and_residue()
    return np.concatenate([imfs.T, res.reshape(-1,1)], axis=1)

# VMD
def variational_mode_decomposition(F, K): # K is the number of modes
    alpha = 2000       # moderate bandwidth constraint  
    tau = 0.            # noise-tolerance (no strict fidelity enforcement)  
    DC = 0             # no DC part imposed  
    init = 1           # initialize omegas uniformly  
    tol = 1e-7  
    u, u_hat, omega = VMD(F, alpha, tau, K, DC, init, tol)  
    return u.T

# SSA
def SSA(F, L, n_take):
    
    N = len(F)
    K = N - L + 1
    X = np.array([F[i:i+L] for i in range(K)]).T # The trajectory matrix
    
    # If rank(X) = 70 (min(L,K)) there would be no problem (the desired case)
    # However, in case of not, we set the rank(X) to be a number d
    d = np.linalg.matrix_rank(X)
    
    U, Sigma, V = np.linalg.svd(X)
    V = V.T
    X_elem = np.array( [Sigma[i] * U[:,[i]]*V.T[i] for i in range(d)] ) 
       
    print(np.round((Sigma**2)[:n_take].sum() / (Sigma**2).sum() * 100, 3))
    
    def X_to_TS(X_i):
        X_rev = X_i[::-1]
        return np.array([X_rev.diagonal(i).mean() for i in range(-X_i.shape[0]+1, X_i.shape[1])])
    
    F_comp_all = [X_to_TS(X_elem[i]) for i in range(d)]
    F_comp_first = F_comp_all[:n_take-1]
    F_comp_noise = [ sum(F_comp_all[n_take-1:]) ]
    F_comp = F_comp_first + F_comp_noise   
    return F_comp, Sigma

In [ ]:
n_data = unit(data)
v_data = [ weist(data,0.1*(i+1)) for i in range(9) ]
n_eemd = ensemble_empirical_mode_decomposition(n_data)
v_eemd = [ensemble_empirical_mode_decomposition(v_data[i]) for i in range(9)]
n_vmd = variational_mode_decomposition(n_data,14)
v_vmd = [variational_mode_decomposition(v_data[i], 14) for i in range(9)]

L = [int(14+2*i) for i in range(10)]
n_take = 14
N_ssa, V_ssa = [], []
for wsize in L:
    n_ssa = SSA(n_data,wsize,n_take)[0]
    v_ssa = [SSA(v_data[i],wsize,n_take)[0] for i in range(9)]
    N_ssa.append(n_ssa)
    V_ssa.append(v_ssa)

N_ssa_mat = [ np.array(N_ssa[i]).T for i in range(10) ]   
V_ssa_mat = []
for i in range(10):
    tem = []
    for j in range(9):
        mat = np.array(V_ssa[i][j]).T
        tem.append(mat)
    V_ssa_mat.append(tem)
    
tem = []
for i in range(9):
    if v_eemd[i].shape[1] == 15:
        tem.append(v_eemd[i][1:,:-1])
    else:
        tem.append(v_eemd[i][1:])  

# n_eemd: Native_EEMD
# v_eemd: CCD applied EEMD
# n_vmd: Native VMD
# v_vmd: CCD applied VMD
# n_ssa: Native SSA
# v_ssa: CCD applied SSA
np.savetxt('n_eemd.txt', n_eemd[1:])
np.savetxt('v_eemd.txt', np.concatenate([tem[i] for i in range(9)]))

np.savetxt('n_vmd.txt', n_vmd)
np.savetxt('v_vmd.txt', np.concatenate(v_vmd))

np.savetxt('n_ssa.txt', np.concatenate([N_ssa_mat[i][1:] for i in range(10)]))
np.savetxt('v_ssa.txt', np.concatenate([ np.concatenate([ V_ssa_mat[i][j][1:] for j in range(9)]) for i in range(10)]))        